In [3]:
# Sam Brown
# Sam_brown@mines.edu
# Load in pos files 

#Imports
import pandas as pd
import matplotlib.pyplot as plt
from pyproj import CRS, Transformer
from concurrent.futures import ThreadPoolExecutor
from typing import Tuple
import numpy as np
from datetime import datetime
import os
from Datastream import Datastream
from collections import defaultdict

# Path
sta_path = "/Users/sambrown04/Documents/SURF/PPP"
stas = ["la01", "la02", "la03", "la04", "la05", "la06", "la07", "la08", "la09",
        "la10", "la11", "la12", "la13", "la14", "la15", "la16", "la17", "la18",
        "ws04", "ws05",
        "gz01", "gz02", "gz03", "gz04", "gz05", "gz06", "gz07", "gz08", "gz09",
        "gz10", "gz11", "gz12", "gz13", "gz14", "gz15", "gz16", "gz17", "gz18",
        "gz19", "gz20",
        "mg01", "mg02", "mg03", "mg04", "mg05", "mg06", "mg07",
        "slw1"
        ]
year_arr = ["2008","2009","2010","2011","2012",'2013','2014','2015',"2016","2017","2018","2019"]
   

In [4]:
def load_station(sta: str, years: list, directory: str) -> Datastream:
    """
    Load a station's worth of data for the years specified.
 
    Parameters
    ----------
    sta: str
        Station name
    years : list
        List of year folders to include in this data stream.

    returns
    -------
    Datastream
    Datastream object containing the data in the position file
    """
    cat = Datastream(os.path.join(directory, sta), sta, years)
    return cat

# Load stations in parallel
with ThreadPoolExecutor() as executor:
    results = executor.map(load_station, stas, [year_arr] * len(stas), [sta_path] * len(stas))

cats = [cat for cat in results if cat is not None]

Matched year 2010 in folder 2010 for station la01
Matched year 2010 in folder 2010 for station la04
Matched year 2010 in folder 2010 for station la06
Matched year 2008 in folder 2008 for station la09
Matched year 2011 in folder 2011 for station la03
Matched year 2010 in folder 2010 for station la05
Matched year 2010 in folder 2010 for station la02
Matched year 2010 in folder 2010 for station la10
Matched year 2010 in folder 2010 for station la08
Matched year 2011 in folder 2011 for station la11
Matched year 2010 in folder 2010 for station la07
Matched year 2012 in folder 2012 for station la12
Matched year 2011 in folder 2011 for station la13
Matched year 2014 in folder 2014 for station la14
Matched year 2012 in folder 2012 for station la13
Matched year 2011 in folder 2011 for station la01
Matched year 2011 in folder 2011 for station la04
Matched year 2011 in folder 2011 for station la10
Matched year 2013 in folder 2013 for station la12
Matched year 2011 in folder 2011 for station la06


/opt/anaconda3/lib/python3.12/site-packages/pyproj/transformer.py:820: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return self._transformer._transform_point(


Matched year 2010 in folder 2010 for station ws04


/opt/anaconda3/lib/python3.12/site-packages/pyproj/transformer.py:820: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return self._transformer._transform_point(


Matched year 2010 in folder 2010 for station ws05
Matched year 2011 in folder 2011 for station gz01
Matched year 2011 in folder 2011 for station gz02
Matched year 2013 in folder 2013 for station la07
Matched year 2011 in folder 2011 for station gz03
Matched year 2013 in folder 2013 for station la08
Matched year 2012 in folder 2012 for station gz02
Matched year 2011 in folder 2011 for station gz04
Matched year 2012 in folder 2012 for station la09
Matched year 2012 in folder 2012 for station gz01
Matched year 2014 in folder 2014 for station la04
Matched year 2014 in folder 2014 for station la10
Matched year 2019 in folder 2019 for station la17
Matched year 2012 in folder 2012 for station gz04
Matched year 2012 in folder 2012 for station gz03
Matched year 2011 in folder 2011 for station gz05
Matched year 2014 in folder 2014 for station la06
Matched year 2013 in folder 2013 for station gz02
Matched year 2013 in folder 2013 for station la02
Matched year 2012 in folder 2012 for station gz05


In [42]:
# Want to make a list of dataframes for the year 2011 where each dataframe corresponds to a station that was up during that time
dfs_by_year = defaultdict(list)
years = range(2013, 2019)

for cat in cats:
    df = cat.data

    if "time" not in df.columns or df.empty:
        continue

    print(cat.name, df["time"].min(), df["time"].max())

    for yr in years:
        start = pd.to_datetime(f"{yr}-01-01")
        end = pd.to_datetime(f"{yr}-12-31 23:59:59")

        df_yr = df[(df["time"] >= start) & (df["time"] <= end)].copy()
        if not df_yr.empty:
            df_yr["station"] = cat.name
            dfs_by_year[yr].append(df_yr)

print("2011:", len(dfs_by_year[2011]), "stations")
# ERROR needs fixing

la01 2010-01-15 00:40:00 2013-12-29 22:30:45
la02 2010-02-05 21:53:00 2019-06-16 23:59:45
la03 2011-01-01 04:18:45 2011-09-12 01:01:15
la04 2010-01-15 06:03:30 2014-12-07 21:44:30
la05 2010-01-19 00:38:15 2013-04-20 07:46:30
la06 2010-01-15 01:36:45 2014-12-04 22:04:45
la07 2010-01-15 03:36:45 2017-11-27 22:51:15
la08 2010-01-19 01:44:15 2017-11-28 23:59:45
la09 2008-01-25 00:00:00 2019-05-01 23:59:45
la10 2010-01-17 01:37:15 2014-12-08 00:07:30
la11 2011-01-01 05:16:45 2011-09-15 23:31:15
la12 2012-01-04 22:28:00 2014-01-07 01:38:15
la13 2011-12-12 01:45:45 2014-01-02 23:59:45
la14 2014-01-05 00:00:00 2018-12-25 23:59:45
la15 2014-01-11 04:11:45 2014-12-07 04:24:30
la16 2014-01-28 01:51:15 2016-12-06 23:32:45
la17 2016-12-12 03:20:45 2019-01-17 23:59:45
la18 2016-12-11 21:40:15 2018-12-30 01:39:15
ws04 2010-12-03 03:22:00 2010-12-30 22:01:00
ws05 2010-12-03 00:18:00 2010-12-30 20:34:45
gz01 2011-01-12 00:00:00 2013-12-31 23:59:45
gz02 2011-01-12 00:00:00 2014-01-21 23:59:45
gz03 2011-

In [43]:
# Save all these to my computer
output_dir = "/Users/sambrown04/Documents/SURF/POS Data"

for year, station_dfs in dfs_by_year.items():
    year_dir = os.path.join(output_dir, str(year))
    os.makedirs(year_dir, exist_ok=True)  # create folder 

    for df in station_dfs:
        if "station" not in df.columns:
            continue  # Skip if no station

        station_name = df["station"].iloc[0]
        file_path = os.path.join(year_dir, f"{station_name}.csv")
        df.to_csv(file_path, index=False)
        print(f"Saved {file_path}")

Saved /Users/sambrown04/Documents/SURF/POS Data/2013/la01.csv
Saved /Users/sambrown04/Documents/SURF/POS Data/2013/la02.csv
Saved /Users/sambrown04/Documents/SURF/POS Data/2013/la04.csv
Saved /Users/sambrown04/Documents/SURF/POS Data/2013/la05.csv
Saved /Users/sambrown04/Documents/SURF/POS Data/2013/la06.csv
Saved /Users/sambrown04/Documents/SURF/POS Data/2013/la07.csv
Saved /Users/sambrown04/Documents/SURF/POS Data/2013/la08.csv
Saved /Users/sambrown04/Documents/SURF/POS Data/2013/la09.csv
Saved /Users/sambrown04/Documents/SURF/POS Data/2013/la10.csv
Saved /Users/sambrown04/Documents/SURF/POS Data/2013/la12.csv
Saved /Users/sambrown04/Documents/SURF/POS Data/2013/la13.csv
Saved /Users/sambrown04/Documents/SURF/POS Data/2013/gz01.csv
Saved /Users/sambrown04/Documents/SURF/POS Data/2013/gz02.csv
Saved /Users/sambrown04/Documents/SURF/POS Data/2013/gz03.csv
Saved /Users/sambrown04/Documents/SURF/POS Data/2013/gz04.csv
Saved /Users/sambrown04/Documents/SURF/POS Data/2013/gz05.csv
Saved /U